Tenemos una colección de documentos (textos) almacenados en un
cluster. Se quiere establecer un ranking de los patrones mas frecuentes
para la aparición de letras en las palabras. Por ejemplo “sister”,
“ejects” , “ninety” y “amazon” responden al patrón “abacde”.
Programar en map-reduce un programa que genere como resultado un
listado de tipo (patron, frecuencia) indicando cuántas veces aparece
cada patrón en la colección de documentos. Usar combiners como
método de agregación.

[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqss6zt4ym)

Creo un rdd con el tres documentos obtenidos de wikipedia. Primero mapeo la funcion `word_tokenizer` que crea una lista de palabras desde un string. Como queremos una lista de palabras, hay que usar `flatMap`.

In [ ]:
import wikipedia as wp
from string import punctuation
from re import sub
wp.set_lang("en")
articulo1 = wp.page("Zimbawe").summary
articulo2 = wp.page("Camboya").summary
articulo3 = wp.page("Djibouti").summary
rdd = sc.parallelize([articulo1,articulo2,articulo3])

In [ ]:
def pattern(word):
    set_letras = {}
    pattern = []
    for i in word:
        if i not in set_letras:
            set_letras[i] = len(set_letras)
        pattern.append(set_letras[i])
    return (tuple(pattern),word)

def word_tokenizer(string):
    return sub('['+punctuation+']',' ',string.replace('\n',' ')).split(' ')

In [ ]:
rdd = rdd.map(word_tokenizer).flatMap(lambda x: x)
rdd = rdd.filter(lambda x: len(x)>0)

Previamente definimos la funcion `pattern` que devuelve una tupla con una secuencia numerica indicando el patron de la palabra y la palabra. La mapeamos al RDD, agrupamos por patron (clave). Eso nos deja un RDD con registros del tipo `<patron, iterable de palabras>`. Queremos la cantidad de palabras por patron, asi que con `len` obtenemos la cantidad.

rdd = rdd.map(pattern)
rdd = rdd.groupByKey()
rdd = rdd.mapValues(lambda x: len(x))
rdd.collect()

Otra forma podria haber sido usando `reduceByKey` para sumar, algo asi:  
```
rdd = rdd.map(pattern)
rdd = rdd.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y)
rdd.collect()
```